# GraphNeT Training Script (P-ONE, OM-center geometry) — Notes & Rationale

I wrote this document to explain my training script end-to-end: what each part does, why I chose this graph representation, what SLURM resources I request, and what the “missing GCD / missing PMT positions” problem changes in practice.

---

## 0) What I am trying to do

I want to train a Graph Neural Network (GraphNeT) to predict **event energy** (`truth["energy"]`) using detector pulse information stored in **Parquet**.

My dataset limitation is important:

- I do **not** have per-PMT positions.
- I only have **Optical Module (OM) center** positions (`dom_x, dom_y, dom_z`).
- Pulses have at least: OM-center position, time, charge.

So I need a graph design that uses geometry but does not create “fake locality” inside a single OM.

---

## 1) High-level pipeline (what happens when I run the script)

When I execute the script, this is the flow:

1. I define a **Detector mapping** (`PONE`) so GraphNeT knows how to interpret input columns.
2. I define a **graph representation** (`KNNGraph`) with OM-level nodes using `ClusterSummaryFeatures`.
3. I create a **ParquetDataset**, which loads events and builds one graph per event.
4. I create a **DataLoader**, which batches graphs for training.
5. I define a **GNN backbone** (`DynEdge`).
6. I define a **task head** (`EnergyReconstruction`) to regress energy.
7. I create a **StandardModel** from (data_representation + backbone + task).
8. I train with `model.fit(...)` on GPU if available.

---

## 2) The GCD / geometry problem in my dataset

### 2.1 What I am missing
Normally, a “good” detector geometry setup (GCD-like information) would let me place each PMT precisely inside an OM. But my dataset does not have that. Instead:

- All pulses from the same OM share the exact same `(dom_x, dom_y, dom_z)` (the OM center).

### 2.2 Why pulse-level KNN is a problem here
If I used:

- `NodesAsPulses()` (node = pulse)
- KNN edges based on `(dom_x, dom_y, dom_z)`

then many pulses would be at identical coordinates (same OM center). That creates a failure mode:

- For a given pulse-node, the “closest K pulses” are often just other pulses from the **same OM** (distance 0).
- So the graph ends up dominated by **within-OM** edges, not **between-OM** edges.
- That reduces real spatial information flow across the detector and can confuse the model.

---

## 3) My solution: OM-level nodes via clustering + KNN between OMs

To avoid the “same OM pulse clique” problem, I switched node construction to:

- `ClusterSummaryFeatures(cluster_on=["dom_x", "dom_y", "dom_z"], ...)`

This makes:

- **One node per OM** (per event), created by clustering pulses that share the same OM-center position.
- Node features become **summary statistics** of the pulses in that OM.

Then I connect these OM nodes using:

- `KNNGraph(nb_nearest_neighbours=K, columns=[0,1,2])`

Meaning:

- For each OM node, I connect it to its **K nearest neighboring OMs** in 3D geometry.

This fits my data limitation: I cannot resolve PMT micro-geometry, but I can still use OM spatial layout.

---

## 4) What `ClusterSummaryFeatures` features mean (the ones I see in config)

My `ClusterSummaryFeatures` configuration builds OM-level node features. Here is what each one means conceptually.

### 4.1 Geometry identity
Because I cluster on `dom_x, dom_y, dom_z`, every node starts with these as features:
- `dom_x, dom_y, dom_z`

These are also used for KNN distance (I set `columns=[0,1,2]`).

### 4.2 Charge features
#### `total_charge`
- Sum of all pulse charges in that OM (a proxy for “how bright” that OM was in the event).
- With `charge_standardization="log"`, the feature is log10-scaled internally.

#### `charge_after_t` (e.g., `charge_after_10ns`, `charge_after_50ns`, `charge_after_100ns`)
- “How much charge has accumulated within **t nanoseconds** after the reference time.”
- These capture the *early vs late* charge buildup shape.
- Also log10-scaled internally due to `charge_standardization="log"`.

### 4.3 Time features
There is a reference time used internally (charge-weighted median time in the cluster), and some time features are expressed relative to it.

#### `time_of_first_hit`
- Time of the first pulse in that OM (then shifted relative to the internal reference time).
- Standardized with `time_standardization=0.001` (a multiplicative scaling).

#### `time_spread`
- “How wide is the pulse time distribution” inside the OM.
- Roughly (last time − first time), depending on implementation.
- Scaled by `0.001`.

#### `time_std`
- Standard deviation of pulse times inside the OM.
- Scaled by `0.001`.

#### `time_after_charge_pctX` (e.g., `time_after_charge_pct50`)
- For a given percentile X, it measures:  
  “How long until the OM accumulated **X% of its total charge**?”
- This describes the temporal charge collection profile.
- Scaled by `0.001`.

### 4.4 Counts
#### `add_counts=True` → `counts`
- Adds the number of pulses in that OM cluster (pulse multiplicity).
- This helps the model distinguish “few big pulses” vs “many small pulses,” etc.

### 4.5 Important standardization note (to avoid double transforms)
Because `ClusterSummaryFeatures` already standardizes:
- charge features (log10)
- time features (multiply by 0.001)

I keep my `PONE._charge()` and `PONE._time()` as identity (no extra log or scaling).  
That avoids double-standardizing the same information.

---

## 5) Graph connectivity: what KNN means in my script

I set:
- `K = 8`
- `columns = [0, 1, 2]`

Because the node feature vector begins with:
- `[dom_x, dom_y, dom_z, ...]`

So KNN distance uses exactly those geometry columns.

Interpretation:

- Each OM node connects to its **8 nearest OM nodes** in 3D space.
- This defines message-passing neighborhood size.
- If I see memory issues or slow graph building, I can reduce K (e.g., 8 → 6 or 4).

---

## 6) Model architecture: DynEdge + EnergyReconstruction

### 6.1 Backbone: `DynEdge`
I use `DynEdge` as the GNN backbone. It performs message passing on the graph and produces an event-level representation.

I set:
- `nb_inputs = data_representation.nb_outputs`  
  So the backbone input dimension matches the OM-node feature dimension produced by `ClusterSummaryFeatures`.

I also set:
- `global_pooling_schemes=["min", "max", "mean"]`  
  This means the model forms event-level features by pooling node embeddings using multiple aggregation operators.

### 6.2 Task head: `EnergyReconstruction`
I use:
- `EnergyReconstruction(target_labels=["energy"])`
- `LogCoshLoss()`

So the model is trained as a regression model predicting event energy.

---

## 7) Training setup in my script

I train with `StandardModel.fit(...)` using a **single GPU** (if CUDA is available).

My key training choices:

- **Epochs**
  - `MAX_EPOCHS = 5`
  - This is a short “baseline run” to confirm the full pipeline works end-to-end.

- **GPU selection**
  - `GPUS = 1 if torch.cuda.is_available() else None`
  - If CUDA is available, training runs on 1 GPU (in my case, a MIG slice).

- **Distributed strategy**
  - `distribution_strategy="auto"`
  - This prevents PyTorch Lightning from trying to use notebook-incompatible DDP settings.
  - Since I only have 1 GPU, I do **not** need DDP anyway.

- **H100 performance tweak**
  - I set:
    ```python
    torch.set_float32_matmul_precision("medium")
    ```
  - This can improve performance on H100 Tensor Cores (without changing my model code).

---

## 8) SLURM resources: what I request and why

Even though the model trains on **GPU**, I still need **CPU** for:
- reading Parquet from disk,
- clustering pulses into OM nodes (`ClusterSummaryFeatures`),
- constructing KNN edges between OM nodes,
- batching in the DataLoader.

If I request too little CPU, the GPU will often **wait** for the next batch.

### 8.1 The `salloc` configuration I plan to use

I will allocate:

```bash
salloc --time=2:00:00 \
  --account=def-nahee \
  --mem=32G \
  --cpus-per-task=4 \
  --gpus-per-node=nvidia_h100_80gb_hbm3_1g.10gb:1


Here is what each flag means:

- `--time=2:00:00`  
  A 2-hour walltime limit for the job.

- `--account=def-nahee`  
  The SLURM account that gets charged for the allocation.

- `--mem=32G`  
  32 GB of CPU RAM for Parquet reading, OM clustering, graph construction, and batching.

- `--cpus-per-task=4`  
  4 CPU cores for the process. I need CPU for the DataLoader + graph building even when training on GPU.  
  This matches a stable baseline like `NUM_WORKERS=2`.

- `--gpus-per-node=nvidia_h100_80gb_hbm3_1g.10gb:1`  
  1 GPU allocation on the node, specifically an H100 MIG slice (1g.10gb), i.e. about ~10 GB VRAM.




### 8.2 How I run the script after `salloc`

After SLURM grants the allocation, I run the script inside the allocated resources using `srun`:



```bash
srun --cpus-per-task=4 python my_training_script.py
```


I use `srun` because it launches the process through SLURM, so resource binding is correct and PyTorch Lightning detects the SLURM environment cleanly.

---

## 9) Practical tuning knobs (if it is slow or if I hit OOM)

### 9.1 If the run feels slow (especially at the start)

It is normal if the progress bar stays at `0/...` for a while at the beginning, because:

- DataLoader worker processes are being created
- the filesystem cache is warming up
- the first event graph is being constructed (clustering + KNN edges)

What I adjust first:

- With `--cpus-per-task=4`, I keep `NUM_WORKERS=2` (stable baseline).
- If I request `--cpus-per-task=8` later, I try `NUM_WORKERS=4`.

If multiprocessing causes issues, I temporarily switch to:

- `NUM_WORKERS=0` (debug mode; often more stable, sometimes slower)

### 9.2 If I get CUDA out-of-memory (MIG 10GB)

With a 10GB MIG slice, out-of-memory is the most likely GPU-side failure mode. I change settings in this order:

1. Reduce batch size: `BATCH_SIZE = 4 -> 2`
2. Reduce edges: `K = 8 -> 6 -> 4`
3. Simplify `ClusterSummaryFeatures` (fewer percentiles, fewer `charge_after_t` times, disable optional time features)

---

## 10) Why this setup is a good baseline for my dataset

Because I only have OM-center geometry (no PMT positions):

- Pulse-level KNN can become degenerate because many pulses share identical `(dom_x, dom_y, dom_z)` and connect mostly within the same OM.
- OM-level clustering (`ClusterSummaryFeatures`) creates one physically meaningful node per OM.
- KNN edges then connect nearby OMs, which is the best spatial structure I can trust with my available geometry.
- DynEdge is a strong baseline GNN backbone for detector-style graphs in GraphNeT.

Overall, this pipeline is stable to debug, physically reasonable under my constraints, and realistic to run on a 10GB MIG slice.


In [1]:
## 0) Imports and Little Helper Function:

import os
from datetime import datetime
from typing import Dict, Callable

import torch
from graphnet.models.detector.detector import Detector

from graphnet.models.data_representation import KNNGraph
from graphnet.models.data_representation.graphs import ClusterSummaryFeatures

from graphnet.models.gnn.dynedge import DynEdge
from graphnet.data.dataset.parquet.parquet_dataset import ParquetDataset
from graphnet.data.dataloader import DataLoader
from graphnet.models.task.reconstruction import EnergyReconstruction
from graphnet.training.loss_functions import LogCoshLoss
from graphnet.models import StandardModel


def section_banner(name: str) -> None:
    print(
        f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] ==> Starting section: {name}",
        flush=True,
    )


section_banner("0) Imports and Helper")


## 0) Global Settings:

section_banner("0) Global Settings")

PARQUET_ROOT = "/project/def-nahee/kbas/POM_Response_Parquet/merged"
PULSEMAPS = "features"   # pulse/hit parquet folder
TRUTH_TABLE = "truth"    # truth parquet folder

TRUTH_LABELS = ["energy"]


## 0) Environment Information:

section_banner("0) Environment Information")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))
    torch.set_float32_matmul_precision("medium")
    print("Set torch matmul precision to: medium")
else:
    print("GPU: (not available)")

print("os.cpu_count:", os.cpu_count())

if hasattr(os, "sched_getaffinity"):
    affinity = sorted(os.sched_getaffinity(0))
    print("affinity cores:", len(affinity))
    print("affinity set (first 20):", affinity[:20])

print("SLURM_CPUS_PER_TASK:", os.environ.get("SLURM_CPUS_PER_TASK"))
print("SLURM_JOB_CPUS_PER_NODE:", os.environ.get("SLURM_JOB_CPUS_PER_NODE"))



## 0) My Classes: PONE (Detector)

section_banner("0) My Classes: PONE (Detector)")


class PONE(Detector):
    """Detector class for P-ONE."""

    xyz = ["dom_x", "dom_y", "dom_z"]
    string_id_column = "string"
    sensor_id_column = "sensor_id"

    def feature_map(self) -> Dict[str, Callable]:
        return {
            "dom_x": self._xyz,
            "dom_y": self._xyz,
            "dom_z": self._xyz,
            "dom_time": self._time,
            "charge": self._charge,
        }

    def _xyz(self, x: torch.Tensor) -> torch.Tensor:
        return x  # x / 500.0

    def _time(self, x: torch.Tensor) -> torch.Tensor:
        return x  # (x - 1.0e4) / 3.0e4

    def _charge(self, x: torch.Tensor) -> torch.Tensor:
        return x  # torch.log10(x)

graphnet [MainProcess] WARNING  2026-01-05 06:11:02 - has_jammy_flows_package - `jammy_flows` not available. Normalizing Flow functionality is missing.
graphnet [MainProcess] WARNING  2026-01-05 06:11:02 - has_icecube_package - `icecube` not available. Some functionality may be missing.
graphnet [MainProcess] WARNING  2026-01-05 06:11:03 - has_km3net_package - `km3net` not available. Some functionality may be missing.
[2026-01-05 06:11:03] ==> Starting section: 0) Imports and Helper
[2026-01-05 06:11:03] ==> Starting section: 0) Global Settings
[2026-01-05 06:11:03] ==> Starting section: 0) Environment Information
Device: cuda
GPU: NVIDIA H100 80GB HBM3 MIG 1g.10gb
Set torch matmul precision to: medium
os.cpu_count: 48
affinity cores: 1
affinity set (first 20): [0]
SLURM_CPUS_PER_TASK: None
SLURM_JOB_CPUS_PER_NODE: 1
[2026-01-05 06:11:03] ==> Starting section: 0) My Classes: PONE (Detector)


In [2]:
## 1) Data Representation (Graph Definition)

section_banner("1) Data Representation (Graph Definition)")

FEATURES = ["dom_x", "dom_y", "dom_z", "dom_time", "charge"]
detector = PONE(replace_with_identity=FEATURES)

K = 8
print(f"KNN configuration: K={K} (node = OM via ClusterSummaryFeatures)")

node_definition = ClusterSummaryFeatures(
    cluster_on=["dom_x", "dom_y", "dom_z"],
    input_feature_names=FEATURES,
    charge_label="charge",
    time_label="dom_time",
    add_counts=True,
)

data_representation = KNNGraph(
    detector=detector,
    node_definition=node_definition,
    nb_nearest_neighbours=K,
    columns=[0, 1, 2],
)

[2026-01-05 06:11:03] ==> Starting section: 1) Data Representation (Graph Definition)
graphnet [MainProcess] INFO     2026-01-05 06:11:03 - PONE.__init__ - Writing log to logs/graphnet_20260105-061103.log
KNN configuration: K=8 (node = OM via ClusterSummaryFeatures)


In [3]:
data_representation

KNNGraph(
  KNNGraph(
  {
      'arguments': {
          'detector': ModelConfig(
  {
      'replace_with_identity': ['dom_x', 'dom_y', 'dom_z', 'dom_time', 'charge'],
  }
  ),
          'node_definition': ModelConfig(
  {
      'cluster_on': ['dom_x', 'dom_y', 'dom_z'],
      'input_feature_names': ['dom_x', 'dom_y', 'dom_z', 'dom_time', 'charge'],
      'charge_label': 'charge',
      'time_label': 'dom_time',
      'total_charge': True,
      'charge_after_t': [10, 50, 100],
      'time_of_first_hit': True,
      'time_spread': True,
      'time_std': True,
      'time_after_charge_pct': [1, 3, 5, 11, 15, 20, 50, 80],
      'charge_standardization': 'log',
      'time_standardization': 0.001,
      'order_in_time': True,
      'add_counts': True,
  }
  ),
          'input_feature_names': None,
          'dtype': torch.float32,
          'perturbation_dict': None,
          'seed': None,
          'nb_nearest_neighbours': 8,
          'columns': [0, 1, 2],
          'distance_as_edge

In [4]:
## 2) Dataset (Parquet -> PyG Data)

section_banner("2) Dataset (Parquet -> PyG Data)")

dataset_kwargs = dict(
    path=str(PARQUET_ROOT),
    pulsemaps=PULSEMAPS,
    truth_table=TRUTH_TABLE,
    features=FEATURES,
    truth=TRUTH_LABELS,
)

print("Dataset config:")
print("  path:", dataset_kwargs["path"])
print("  pulsemaps:", dataset_kwargs["pulsemaps"])
print("  truth_table:", dataset_kwargs["truth_table"])
print("  features:", dataset_kwargs["features"])
print("  truth:", dataset_kwargs["truth"])

dataset = ParquetDataset(**dataset_kwargs, data_representation=data_representation)
print("Dataset created.")


[2026-01-05 06:11:03] ==> Starting section: 2) Dataset (Parquet -> PyG Data)
Dataset config:
  path: /project/def-nahee/kbas/POM_Response_Parquet/merged
  pulsemaps: features
  truth_table: truth
  features: ['dom_x', 'dom_y', 'dom_z', 'dom_time', 'charge']
  truth: ['energy']
Dataset created.


In [5]:
## 3) DataLoader

section_banner("3) DataLoader")

BATCH_SIZE = 4
NUM_WORKERS = 2

print(f"DataLoader config: batch_size={BATCH_SIZE}, num_workers={NUM_WORKERS}, shuffle=True")

train_loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=True,
)


[2026-01-05 06:11:03] ==> Starting section: 3) DataLoader
DataLoader config: batch_size=4, num_workers=2, shuffle=True


/project/6061446/kbas/pone_offline/graphnet_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 2 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [6]:
## 4) Backbone

section_banner("4) Backbone")

print("Backbone: DynEdge")
backbone = DynEdge(
    nb_inputs=data_representation.nb_outputs,
    global_pooling_schemes=["min", "max", "mean"],
)



[2026-01-05 06:11:03] ==> Starting section: 4) Backbone
Backbone: DynEdge


In [7]:

## 5) Task: Energy Reconstruction (event-level)

section_banner("5) Task: Energy Reconstruction")

ENERGY_LABEL = "energy"
print(f"Task: EnergyReconstruction (target_labels={[ENERGY_LABEL]})")

task = EnergyReconstruction(
    hidden_size=backbone.nb_outputs,
    target_labels=[ENERGY_LABEL],
    loss_function=LogCoshLoss(),
)


[2026-01-05 06:11:03] ==> Starting section: 5) Task: Energy Reconstruction
Task: EnergyReconstruction (target_labels=['energy'])


In [8]:
## 6) StandardModel

section_banner("6) StandardModel")

print("Building StandardModel ...")
model = StandardModel(
    tasks=[task],
    data_representation=data_representation,
    backbone=backbone,
)
print("Model created.")


[2026-01-05 06:11:03] ==> Starting section: 6) StandardModel
Building StandardModel ...
Model created.


In [ ]:


## 7) Train

section_banner("7) Train")

MAX_EPOCHS = 5
GPUS = 1 if torch.cuda.is_available() else None
print(f"Starting training: max_epochs={MAX_EPOCHS}, gpus={GPUS}")

model.fit(
    train_dataloader=train_loader,
    val_dataloader=None,
    max_epochs=MAX_EPOCHS,
    gpus=GPUS,
    distribution_strategy="auto",
)


[2026-01-05 06:11:03] ==> Starting section: 7) Train
Starting training: max_epochs=5, gpus=1
graphnet [MainProcess] INFO     2026-01-05 06:11:03 - StandardModel._print_callbacks - Training initiated with callbacks: ProgressBar


/project/6061446/kbas/pone_offline/graphnet_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /project/6061446/kbas/pone_offline/graphnet_env/lib/ ...
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
/project/6061446/kbas/pone_offline/graphnet_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `log

Training: |                                                                                                   …